In [16]:
import pandas as pd
import numpy as np
import pickle
import re
import requests
from bs4 import BeautifulSoup

In [2]:
users = pd.read_fwf("zee-users.dat",encoding='ISO-8859-1')

In [3]:
users

,UserID::Gender::Age::Occupation::Zip-code
0,1::F::1::10::48067
1,2::M::56::16::70072
2,3::M::25::15::55117
3,4::M::45::7::02460
4,5::M::25::20::55455
...,...
6035,6036::F::25::15::32603
6036,6037::F::45::1::76006
6037,6038::F::56::1::14706
6038,6039::F::45::0::01060


In [4]:
cols = users.columns.str.split("::")[0]

users = users.iloc[:,0].str.split("::",expand = True)

users.columns = cols

users

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [5]:
movies = pd.read_fwf("zee-movies.dat",encoding='ISO-8859-1')

In [6]:
movies.drop(columns=["Unnamed: 1","Unnamed: 2"],inplace=True)

In [7]:
cols = movies.columns.str.split("::")[0]

movies = movies.iloc[:,0].str.split("::",expand = True)

movies.columns = cols

movies

,Movie ID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [8]:
rating = pd.read_fwf("zee-ratings.dat",encoding='ISO-8859-1')

In [9]:
cols = rating.columns.str.split("::")[0]

rating = rating.iloc[:,0].str.split("::",expand = True)

rating.columns = cols

rating

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [10]:
df = pd.merge(rating,movies,left_on="MovieID",right_on="Movie ID")

In [11]:
df = df.merge(users,left_on="UserID",right_on = "UserID")

In [12]:
df.drop(columns="Movie ID",inplace = True)

In [13]:
df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [14]:
df.shape

(1000209, 10)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1000209 non-null  object
 1   MovieID     1000209 non-null  object
 2   Rating      1000209 non-null  object
 3   Timestamp   1000209 non-null  object
 4   Title       1000209 non-null  object
 5   Genres      996144 non-null   object
 6   Gender      1000209 non-null  object
 7   Age         1000209 non-null  object
 8   Occupation  1000209 non-null  object
 9   Zip-code    1000209 non-null  object
dtypes: object(10)
memory usage: 76.3+ MB


In [43]:
def scrape_genere(name):
    try:
        url = f"https://www.google.com/search?q={name}"
        response=requests.get(url)
        soup = BeautifulSoup(response.content,"html.parser")
        result = soup.find_all("div",class_="BNeawe tAd8D AP7Wnd")[0]
        text = result.get_text()
        st=re.findall("\D{3,}",text)[0]
        st = st.replace("‧","").replace(" ","").replace("/","|")
        return st
    except:
        return name
#     print(text,type(text))

In [44]:
movies[movies["Genres"].isna()]["Title"].apply(scrape_genere)

62                                          Comedy|Action
197                                       Musical|Romance
359     Wonderful, Horrible Life of Leni Riefenstahl, ...
464                                        Comedy|Romance
720                                      Thriller|Mystery
740     Dr. Strangelove or: How I Learned to Stop Worr...
783     My Life and Times With Antonin Artaud (En comp...
787     Old Lady Who Walked in the Sea, The (Vieille q...
1134                                       Crime|Thriller
1160                                      Fantasy|Mystery
1341    Garden of Finzi-Contini, The (Giardino dei Fin...
1950                                              Viewall
2170    Swept Away (Travolti da un insolito destino ne...
2516                                      Romance|Mystery
2522                                      Musical|Romance
2563                                       Fantasy|Comedy
2721    Final Conflict, The (a.k.a. Omen III: The Fina...
3001          

In [30]:
scrape_genere("Don't Be a Menace to South Central While Drinking Your Juice in the")

'Comedy|Action'

In [39]:
movies[movies["Genres"].isna()].loc[359,"Title"]

'Wonderful, Horrible Life of Leni Riefenstahl, The (Die Macht der B'

In [45]:
"Don't Be a Menace to South Central While Drinking Your Juice in the".replace(" ","%")

"Don't%Be%a%Menace%to%South%Central%While%Drinking%Your%Juice%in%the"

In [77]:
name = "Don't Be a Menace to South Central While Drinking Your Juice in the".replace(" ","+") 

In [89]:
url = f"https://www.imdb.com/find/?q={name}"

In [90]:
# url = "https://www.imdb.com/find?q=Toy+Story"

In [91]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

In [92]:
response = requests.get(url,headers = headers)

In [93]:
response

<Response [200]>

In [94]:
soup = BeautifulSoup(response.content,"html.parser")

In [99]:
div = soup.find_all("div",class_="ipc-metadata-list-summary-item__tc")[0]

In [104]:
h = div.find_all("a")[0]

In [105]:
h

<a aria-disabled="false" class="ipc-metadata-list-summary-item__t" href="/title/tt0116126/?ref_=fn_al_tt_1" role="button" tabindex="0">Don't Be a Menace to South Central While Drinking Your Juice in the Hood</a>

In [109]:
href = "https://www.imdb.com/"+ str(h.get("href"))

In [113]:
response = requests.get(href,headers=headers)

In [114]:
response

<Response [200]>

In [115]:
soup = BeautifulSoup(response.content,"html.parser")

In [120]:
r = soup.find_all("div",class_ = "ipc-chip-list__scroller")[0]

In [131]:
l = []
for i in r.find_all("span"):
    k = i.get_text()
    l.append(k)
    print("|".join(l))

Comedy
Comedy|Crime


In [82]:
def construct_imdb_url(movie_title):
    # Replace spaces with '+' characters and encode special characters
    encoded_title = '+'.join(movie_title.split())
    # Construct the IMDb search URL
    imdb_url = f"https://www.imdb.com/find?q={encoded_title}"
    return imdb_url

# Example usage
movie_title = "Toy Story"
imdb_url = construct_imdb_url(movie_title)
print(imdb_url)


https://www.imdb.com/find?q=Toy+Story
